* Course: YSC4103 MCS Capstone
* Date created: 2019/02/25
* Name: Linfan XIAO
* Description: Implement the Fokas method as described in "Evolution PDEs and augmented eigenfunctions. Finite interval."



# Importing packages

In [3]:
using SymPy
# using Roots
using Distributions
# using IntervalArithmetic
# using IntervalRootFinding
using ApproxFun

# Global variables

In [2]:
tol = 1e-05

1.0e-5

# Helper functions

## `check_all`

Check whether all elements in a not necessarily homogeneous array satisfy a given condition.

Arguments:
* `array`: a generic array, not necessarily homogeneous.
* `condition`: a boolean function to be applied to each element in the array.

In [ ]:
function check_all(array, condition)
    for x in array
        if !condition(x)
            return false
        end
    end
    return true
end

## `set_tol`

Set an appropriate tolerance for checking whether two numbers are approximately equal.

Arguments:
* `x`: A number.
* `y`: A number.

In [ ]:
function set_tol(x::Number, y::Number)
    return tol * mean([x y]) # tol is global variable
end

## `set_tol_matrix`

Set an appropriate tolerance for checking whether two matrices are approximately equal.

Arguments:
* `A`: A matrix with numeric entries.
* `B`: A matrix with numeric entries.

In [ ]:
function set_tol_matrix(A::Array, B::Array)
    if size(A) != size(B)
        throw(error("Matrix dimensions do not match"))
    end
    # Avoid InexactError() when taking norm()
    A = convert(Array{Complex}, A)
    B = convert(Array{Complex}, B)
    return tol * (norm(A,2) + norm(B,2))
end

## `evaluate`

Evaluate a function on a given value.

Arguments:
* `func`: A function of type Julia `Function`, `SymPy.Sym` (absorbed into `Number`), or `Number`.
* `x`: A value on which the function is to be evaluated on.
* `t`: The free symbol in `func` if `func` is a `SymPy.Sym` object, i.e., a symbolic expression.

In [ ]:
function evaluate(func::Union{Function,Number}, x::Number, t = nothing)
    if isa(func, Function)
        return func(x)
    elseif isa(func, SymPy.Sym) # SymPy.Sym must come before Number because SymPy.Sym will be recognized as Number
        return subs(func, t, x)
    else
        return func
    end
end

## `partition`

Generate two-integer partitions ($0$ included) of a given positive integer.

Arguments:
* `n`: A non-negative integer.

In [8]:
function partition(n::Int)
    if n < 0
        throw("Non-negative n required")
    end
    output = []
    for i = 0:n
        j = n - i
        push!(output, (i,j))
    end
    return output
end

partition (generic function with 1 method)

## `symDeriv`

Construct the symbolic expression for the $k$th derivative of a univariate function.

Arguments:
* `u`: A `SymPy.Sym` object that is a symbolic expression in `t`.
* `t`: The free symbol in `u`.
* `k`: The degree of the desired derivative.

In [11]:
function symDeriv(u::SymPy.Sym, t::SymPy.Sym, k::Int)
    if k < 0
        throw(error("Non-negative degree required"))
    end
    y = u
    for i = 1:k
        newY = diff(y, t)
        y = newY
    end
    return y
end

2-element Array{Any,1}:
 (0, 1)
 (1, 0)

## `add_func`

Function addition given by $(f + g)(x) := f(x) + g(x)$.

Arguments:
* `f`: A `Function` or `Number` (convenient representation of constant function).
* `g`: A `Function` or `Number` (convenient representation of constant function).

In [ ]:
function add_func(f::Union{Number, Function}, g::Union{Number, Function})
    function h(x)
        if isa(f, Number)
            if isa(g, Number)
                return f + g
            else
                return f + g(x)
            end
        elseif isa(f, Function)
            if isa(g, Number)
                return f(x) + g
            else
                return f(x) + g(x)
            end
        end
    end
    return h
end

## `mult_func`

Function multiplication given by $(f \cdot g)(x) := f(x) \cdot g(x)$.

Arguments:
* `f`: A `Function` or `Number` (convenient representation of constant function).
* `g`: A `Function` or `Number` (convenient representation of constant function).

In [ ]:
function mult_func(f::Union{Number, Function}, g::Union{Number, Function})
    function h(x)
        if isa(f, Number)
            if isa(g, Number)
                return f * g
            else
                return f * g(x)
            end
        elseif isa(f, Function)
            if isa(g, Number)
                return f(x) * g
            else
                return f(x) * g(x)
            end
        end
    end
    return h
end

## `evaluate_matrix`

Evaluate a matrix of univariate functions at a given value.

Arguments:
* `matrix`: A matrix of univariate functions, where each entry is of type Julia `Function`, `SymPy.Sym` (absorbed into `Number`), or `Number`.
* `a`: A `Number` at which the matrix of functions will be evaluated.
* `t`: The free symbol in the functions that are the matrix's entries if they are `SymPy.Sym` objects.

In [ ]:
function evaluate_matrix(matrix::Array, a::Number, t = nothing)
    (m, n) = size(matrix)
    matrixA = Array{Number}(m,n)
    for i = 1:m
        for j = 1:n
            matrixA[i,j] = evaluate(matrix[i,j], a, t)
        end
    end
    return matrixA
end

## `get_polynomial`

Construct a polynomial 
$$a_nx^n + a_{n-1}x^{n-1} + \cdots + a_1 x + a_0$$
as a Julia `Function` from an array $[a_n, a_{n-1}, \ldots, a_1, a_0]$.

Arguments:
* `coeffList`: An array of numbers that are the coefficients of $x^n, x^{n-1}, \ldots, x, 1$ in the polynomial.

In [ ]:
function get_polynomial(coeffList::Array)
    polynomial = 0
    n = length(coeffList)-1
    for i in 0:n
        newTerm = t -> coeffList[i+1] * t^(n-i)
        polynomial = add_func(polynomial, newTerm)
    end
    return polynomial
end

## `get_polynomialDeriv`

Get the $k$th derivative of a polynomial with known coefficients.

Arguments:
* `coeffList`: An array of numbers that are the coefficients of $x^n, x^{n-1}, \ldots, x, 1$ in the polynomial.
* `k`: The degree of the desired derivative.

In [ ]:
function get_polynomialDeriv(coeffList::Array, k::Int)
    if k < 0
        throw(error("Only nonnegative degrees are allowed"))
    elseif k == 0
        newCoeffList = coeffList
    else
        for counter = 1:k
            n = length(coeffList)
            newCoeffList = hcat([0],[(n-i)*coeffList[i] for i in 1:(n-1)]')
            coeffList = newCoeffList
        end
    end
    return get_polynomial(newCoeffList)
end

# Structs

## `StructDefinitionError`

A struct definition error type is the class of all errors in struct definitions.

In [ ]:
struct StructDefinitionError <: Exception
    msg::String
end

## `SymLinearDifferentialOperator`

A symbolic linear differential operator of order $n$ is encoded by an $1 \times (n+1)$ array of symbolic expressions with at most one free symbol and an interval $[a,b]$.

In [ ]:
struct SymLinearDifferentialOperator
    # Entries in the array should be SymPy.Sym or Number. SymPy.Sym seems to be a subtype of Number, i.e., Array{Union{Number,SymPy.Sym}} returns Array{Number}. But specifying symPFunctions as Array{Number,2} gives a MethodError when the entries are Sympy.Sym objects.
    symPFunctions::Array
    interval::Tuple{Number,Number}
    t::SymPy.Sym
    SymLinearDifferentialOperator(symPFunctions::Array, interval::Tuple{Number,Number}, t::SymPy.Sym) =
    try
        symL = new(symPFunctions, interval, t)
        check_symLinearDifferentialOperator_input(symL)
        return symL
    catch err
        throw(err)
    end
end

function check_symLinearDifferentialOperator_input(symL::SymLinearDifferentialOperator)
    symPFunctions, (a,b), t = symL.symPFunctions, symL.interval, symL.t
    for symPFunc in symPFunctions
        if isa(symPFunc, SymPy.Sym)
            if size(free_symbols(symPFunc)) != (1,) && size(free_symbols(symPFunc)) != (0,)
                throw(StructDefinitionError(:"Only one free symbol is allowed in symP_k"))
            end
        elseif !isa(symPFunc, Number)
            throw(StructDefinitionError(:"symP_k should be SymPy.Sym or Number"))
        end
    end
    return true
end

# Construct adjoint boundary conditions

Algorithm to construct a valid adjoint boundary condition from a given (homogeneous) boundary condition based on Chapter 11 in Theory of Ordinary Differential Equations (Coddington & Levinson). The implementation uses Julia functions as main objects but supports symbolic expressions.

# Approximate roots of exponential polynomial

# The Fokas Transform pair